<a href="https://colab.research.google.com/github/alixintong/btt-ai-know-your-customer/blob/jess/bounding_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# this cell allows access to our folder

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_path = '/content/gdrive/MyDrive/BTTAI_AmericanExpress1'

Mounted at /content/drive


In [8]:
# cell for imports - add as needed
import torch 
import cv2
import numpy as np 
import pandas as pd
import os, os.path
import re
from fastai.vision.all import *

In [3]:
# for reference
#   - fastai segmentation: https://walkwithfastai.com/Segmentation
#   - fastai object detection: https://walkwithfastai.com/Object_Detection

In [9]:
path = '/content/drive/MyDrive/BTTAI_AmericanExpress1/datasets/sampleDataset/frames'

files = get_image_files(path + "/datasheet001_frames")
len(files)

214

In [14]:
# import bounding box datasheet csv
# use datasheet001 as csv instead of xml
url='https://drive.google.com/file/d/1lUoBf1rwPXPlI9Zg2lDzwV4ODtsGJ4u2/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.head()

,point/0/_name,point/0/_x,point/0/_y,point/1/_name,point/1/_x,point/1/_y,point/2/_name,point/2/_x,point/2/_y,point/3/_name,point/3/_x,point/3/_y,_index,_rejected
0,bl,716.625,805.609,tl,737.377,201.383,tr,1180.71,199.127,br,1259.02,791.942,1,False
1,bl,714.326,806.309,tl,736.330,202.300,tr,1179.74,200.028,br,1257.21,792.161,2,False
2,bl,713.031,804.811,tl,734.431,201.449,tr,1177.81,199.795,br,1255.42,791.291,3,False
3,bl,713.845,802.659,tl,734.423,198.474,tr,1177.79,196.852,br,1256.72,789.631,4,False
4,bl,713.088,802.830,tl,734.675,195.678,tr,1179.99,194.057,br,1259.35,790.289,5,False


In [15]:
df["document"] = df["_rejected"].apply(lambda x: not x)

In [17]:
df = df.drop('_rejected', axis = 1)
df.head()

,point/0/_name,point/0/_x,point/0/_y,point/1/_name,point/1/_x,point/1/_y,point/2/_name,point/2/_x,point/2/_y,point/3/_name,point/3/_x,point/3/_y,_index,document
0,bl,716.625,805.609,tl,737.377,201.383,tr,1180.71,199.127,br,1259.02,791.942,1,True
1,bl,714.326,806.309,tl,736.330,202.300,tr,1179.74,200.028,br,1257.21,792.161,2,True
2,bl,713.031,804.811,tl,734.431,201.449,tr,1177.81,199.795,br,1255.42,791.291,3,True
3,bl,713.845,802.659,tl,734.423,198.474,tr,1177.79,196.852,br,1256.72,789.631,4,True
4,bl,713.088,802.830,tl,734.675,195.678,tr,1179.99,194.057,br,1259.35,790.289,5,True


In [18]:
def get_coordinates(row_data):
  x1 = min(row_data[1], row_data[4], row_data[7], row_data[10])
  y1 = min(row_data[2], row_data[5], row_data[8], row_data[11])
  x2 = max(row_data[1], row_data[4], row_data[7], row_data[10])
  y2 = max(row_data[2], row_data[5], row_data[8], row_data[11])
  bb_cd = [x1, y1, x2, y2] 
  return bb_cd

In [19]:
bb = []
bb.append(df.apply(lambda row: get_coordinates(row), axis = 1))

In [25]:
bb = np.array(bb)
bb = bb.flatten()

In [30]:
bb = list(bb)

In [31]:
print(bb)

[[716.625, 199.127, 1259.02, 805.609], [714.326, 200.028, 1257.21, 806.309], [713.031, 199.795, 1255.42, 804.811], [713.845, 196.852, 1256.72, 802.659], [713.088, 194.057, 1259.35, 802.83], [710.712, 192.17, 1259.97, 804.627], [708.63, 190.434, 1260.92, 805.288], [709.185, 188.701, 1264.0, 805.897], [707.649, 185.794, 1265.47, 806.049], [704.386, 182.308, 1266.23, 806.904], [706.751, 182.023, 1262.08, 806.144], [710.254, 194.128, 1254.57, 801.102], [713.1, 194.844, 1256.77, 801.072], [714.904, 195.971, 1258.58, 801.413], [714.898, 196.899, 1257.45, 801.527], [713.67, 197.032, 1255.59, 801.477], [713.323, 197.865, 1255.11, 800.823], [714.12, 197.717, 1254.63, 799.75], [712.307, 196.862, 1252.97, 799.758], [709.416, 197.829, 1250.41, 800.245], [709.243, 196.886, 1249.75, 799.384], [708.073, 195.934, 1248.59, 798.56], [706.282, 195.617, 1246.77, 797.391], [708.759, 194.719, 1248.76, 796.698], [710.111, 194.665, 1250.61, 796.567], [715.25, 195.653, 1255.14, 796.404], [719.258, 195.622, 125

In [32]:
df_bb = pd.DataFrame(data=bb, columns=['x1', 'y1', 'x2', 'y2'])

In [33]:
df_bb.head()

,x1,y1,x2,y2
0,716.625,199.127,1259.02,805.609
1,714.326,200.028,1257.21,806.309
2,713.031,199.795,1255.42,804.811
3,713.845,196.852,1256.72,802.659
4,713.088,194.057,1259.35,802.830


In [34]:
df_bb['document'] = df['document']
df_bb.head() # df_bb would be the labels of bounding boxes

,x1,y1,x2,y2,document
0,716.625,199.127,1259.02,805.609,True
1,714.326,200.028,1257.21,806.309,True
2,713.031,199.795,1255.42,804.811,True
3,713.845,196.852,1256.72,802.659,True
4,713.088,194.057,1259.35,802.830,True
